## MÓDULO 2. EVALUACIÓN 2

**OBJETIVO**: identificar todas las universidades ubicadas en tres países específicos: Estados Unidos, Canadá y Argentina.

In [21]:
import requests
import pandas as pd
import numpy as np
import mysql.connector
from geopy.geocoders import Nominatim


import warnings
warnings.filterwarnings('ignore')

1. Utilizando la API extraed toda la información que podáis de ella. La url para hacer las llamadas es:

In [22]:
def llamar_api():
    
    """
    Realiza llamadas a una API para obtener información de universidades en diferentes países.
    
    Returns:
        pd.DataFrame: DataFrame con la información de universidades.
    """
    
    df_universidades = pd.DataFrame() #creamos un dataframe vacío en el que ir sumando cada país
    
    paises = ["United States", "Canada", "Argentina"]
    
    for pais in paises:
        response = requests.get(url= f"http://universities.hipolabs.com/search?country={pais}")
        print(f"Resultado de la conexión de {pais}: {response.status_code} {response.reason}")
        df = pd.json_normalize(response.json())
        df_universidades = pd.concat([df_universidades, df], axis = 0)
    return df_universidades
        

In [23]:
df_universidades = llamar_api()

df_universidades.head(5)

Resultado de la conexión de United States: 200 OK
Resultado de la conexión de Canada: 200 OK
Resultado de la conexión de Argentina: 200 OK


,domains,country,name,web_pages,state-province,alpha_two_code
0,[marywood.edu],United States,Marywood University,[http://www.marywood.edu],None,US
1,[lindenwood.edu],United States,Lindenwood University,[http://www.lindenwood.edu/],None,US
2,[sullivan.edu],United States,Sullivan University,[https://sullivan.edu/],None,US
3,[fscj.edu],United States,Florida State College at Jacksonville,[https://www.fscj.edu/],None,US
4,[xavier.edu],United States,Xavier University,[https://www.xavier.edu/],None,US


In [24]:
df_universidades.duplicated(subset= "name").sum()

2517

___
2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:
    - Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.
    - La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.

3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejericio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva. Aquí tenéis la documentación de este método.

In [25]:
# Creamos una función con todo el proceso de limpieza:

def limpiar_columnas(dataframe= df_universidades):
    
    """
    Limpia y transforma las columnas de un DataFrame de universidades.

    Args:
        dataframe (pd.DataFrame, optional): DataFrame con información de universidades. Por defecto es df_universidades.

    Returns:
        pd.DataFrame: DataFrame con las columnas limpias y transformadas.
    """
    
    dataframe.columns = [column.replace("-", "_") for column in dataframe.columns]
    dataframe.drop("domains", axis= 1, inplace= True)
    dataframe = dataframe.explode("web_pages")
    return dataframe

In [26]:
df_universidades = limpiar_columnas()
df_universidades.head(2)

,country,name,web_pages,state_province,alpha_two_code
0,United States,Marywood University,http://www.marywood.edu,None,US
1,United States,Lindenwood University,http://www.lindenwood.edu/,None,US


___
4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

In [27]:
df_universidades.duplicated(subset= "name").sum()

2543

In [28]:
df_universidades.shape

(5034, 5)

In [29]:
df_universidades.drop_duplicates(subset= "name", inplace= True)

In [30]:
df_universidades.shape

(2491, 5)

Lo óptimo sería que este paso esté también dentro de la función de limpieza anterior, pero los siguientes pasos del ejercicios son también de limpieza. Al finalizar todos estos ejercicios, crearé una nueva función que aúne todos los pasos de limpieza (no lo hago ahora para que se mantengan en el ejercicio las comprobaciones realizadas).

___
5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [31]:
def cambiar_none(valor):
    
    """
    Convierte un valor igual a None en un valor NaN (Not a Number) de NumPy.

    Args:
        valor: El valor a ser evaluado y posiblemente reemplazado.

    Returns:
        valor: El valor reemplazado por NaN si era None, o el mismo valor si no era None.
    """
    
    if valor == None:
        return np.nan
    else:
        return valor

In [32]:
df_universidades["state_province"] = df_universidades["state_province"].apply(cambiar_none)

In [33]:
# Otra manera más eficiente de hacerlo (visto en la evaluación):
df_universidades["state_province"] = df_universidades["state_province"].apply(lambda valor: np.nan if valor == None else valor)

In [34]:
df_universidades.sample(10)

,country,name,web_pages,state_province,alpha_two_code
1877,United States,Stanly Community College,http://www.stanly.edu,NaN,US
1537,United States,Iowa Central Community College,http://www.iowacentral.edu,NaN,US
65,United States,University of Mississippi Medical Center,http://www.umc.edu/,NaN,US
1756,United States,County College of Morris,https://www.ccm.edu/,NaN,US
395,United States,Fayetteville State University,http://www.uncfsu.edu/,NaN,US
823,United States,Slippery Rock University of Pennsylvania,http://www.sru.edu/,Pennsylvania,US
38,Argentina,Universidad Empresarial Siglio 21,http://www.uesiglo21.edu.ar/,Córdoba,AR
235,United States,"California State University, Los Angeles",http://www.calstatela.edu/,NaN,US
2042,United States,Southwest Texas Junior College,http://www.swtjc.net,NaN,US
1960,United States,University of Pittsburgh-​Titusville,http://www.upt.pitt.edu,Pennsylvania,US


No está en el ejercicio, pero para poder insertar los datos en SQL, vamos al limpiar los nombres de las universidades que tienen comillas.

In [35]:
df_universidades[df_universidades["name"].str.contains('"')]

,country,name,web_pages,state_province,alpha_two_code
10,Argentina,"Universidad Argentina ""John F. Kennedy""",http://www.kennedy.edu.ar/,Buenos Aires,AR
41,Argentina,"Universidad ""Juan Agustín Maza""",http://www.umaza.edu.ar/,NaN,AR
69,Argentina,"Universidad Nacional de la Patagonia ""San Juan...",http://www.unp.edu.ar/,NaN,AR
79,Argentina,"Universidad del Norte ""Santo Tomás de Aquino""",http://www.unsta.edu.ar/,NaN,AR


In [36]:
df_universidades["name"] = df_universidades["name"].str.replace('"', '')

In [37]:
df_universidades["name"].str.contains('"').sum()

0

___
6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.

In [38]:
df_universidades["state_province"].isnull().sum()

2204

In [39]:
df_universidades["state_province"].fillna("unknown", inplace= True)

In [40]:
df_universidades["state_province"].isnull().sum()

0

___
7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso, aquí la documentación. Para desarrollar este ejercicio deberéis:
- Sacar los valores únicos de la columna state_province.

In [41]:
df_universidades["state_province"].value_counts()

state_province
unknown                            2204
Ontario                              44
British Columbia                     33
Pennsylvania                         31
Quebec                               24
Buenos Aires                         20
Ciudad Autónoma de Buenos Aires      19
Alberta                              12
NY                                    8
New Brunswick                         7
Nova Scotia                           7
Manitoba                              6
California                            5
Córdoba                               4
New York, NY                          4
Saskatchewan                          4
Texas                                 4
Entre Ríos                            3
Santa Fé                              3
Mendoza                               3
North Carolina                        2
TX                                    2
Ohio                                  2
GA                                    2
CA                       

- Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:
    - NV: reemplazalo por Nevada
    - TX: reemplazalo por Texas
    - IN: reemplazalo por Indianapolis
    - CA: reemplazalo por California
    - VA: reemplazalo por Virginia
    - NY: reemplazalo por New York
    - MI: reemplazalo por Michigan
    - GA: reemplazalo por Georgia
    - ND: reemplazalo por North Dakota

- Otros valores que tenemos más formateados son y que deberemos reemplazar:
    - New York, NY. Deberéis reemplazarlo por "New York".
    - 'Buenos Aires', 'Ciudad Autónoma de Buenos Aires'. En este caso deberéis poner en ambos casos "Buenos Aires"

In [42]:
lista_estados = df_universidades["state_province"].value_counts().index

In [125]:
reemplazo_siglas = [estado.replace("NV", "Nevada").replace("TX", "Texas").replace("IN", "Indianapolis").replace("CA", "California").replace("VA", "Virginia").replace("New York, NY", "New York").replace("NY", "New York").replace("MI", "Michigan").replace("GA", "Georgia").replace("ND", "North Dakota").replace("Ciudad Autónoma de Buenos Aires", "Buenos Aires")
                    for estado in lista_estados]

In [126]:
dict_siglas = dict(zip(lista_estados, reemplazo_siglas))

In [127]:
print(dict_siglas)

{'unknown': 'unknown', 'Ontario': 'Ontario', 'British Columbia': 'British Columbia', 'Pennsylvania': 'Pennsylvania', 'Quebec': 'Quebec', 'Buenos Aires': 'Buenos Aires', 'Ciudad Autónoma de Buenos Aires': 'Buenos Aires', 'Alberta': 'Alberta', 'NY': 'New York', 'New Brunswick': 'New Brunswick', 'Nova Scotia': 'Nova Scotia', 'Manitoba': 'Manitoba', 'California': 'California', 'Córdoba': 'Córdoba', 'New York, NY': 'New York', 'Saskatchewan': 'Saskatchewan', 'Texas': 'Texas', 'Entre Ríos': 'Entre Ríos', 'Santa Fé': 'Santa Fé', 'Mendoza': 'Mendoza', 'North Carolina': 'North Carolina', 'TX': 'Texas', 'Ohio': 'Ohio', 'GA': 'Georgia', 'CA': 'California', 'Santiago Del Estero': 'Santiago Del Estero', 'Salta': 'Salta', 'Washington': 'Washington', 'Misiones': 'Misiones', 'Catamarca': 'Catamarca', 'Yukon': 'Yukon', 'Formosa': 'Formosa', 'Jujuy': 'Jujuy', 'La Rioja': 'La Rioja', 'La Pampa': 'La Pampa', 'San Juan': 'San Juan', 'San Luis': 'San Luis', 'Prince Edward Island': 'Prince Edward Island', 'M

In [128]:
df_universidades["state_province"] = df_universidades["state_province"].map(dict_siglas, na_action="ignore")

In [44]:
# Otra manera más eficiente de hacerlo:

reemplazos = {'NV':'Nevada','VA':'Virginia', 'TX':'Texas','IN':'Indianapolis','CA':'California','NY':'New York', 'ND':'North Dakota', 'MI':'Michigan',
       'GA':'Georgia', 'New York, NY':'New York','Ciudad Autónoma de Buenos Aires':'Buenos Aires'}

df_universidades["state_province"] = df_universidades["state_province"].replace(reemplazos)

___
- Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.

In [45]:
lista_estados_ok = df_universidades["state_province"].unique()
lista_estados_ok

array(['unknown', 'Pennsylvania', 'Texas', 'Utah', 'Nevada', 'Iowa',
       'Virginia', 'Indiana', 'Colorado', 'Ohio', 'Indianapolis',
       'New York', 'California', 'Illinois', 'New Hampshire',
       'North Carolina', 'South Carolina', 'Washington', 'Missouri',
       'North Dakota', 'Michigan', 'Florida', 'Georgia', 'Maine',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza',
       'Santa Fé', 'Santiago Del Estero', 'Misiones', 'Catamarca',
       'Formosa', 'Jujuy', 'La Rioja', 'La Pampa', 'San Juan', 'San Luis',
       'Tucumán'], dtype=object)

___
- Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.

In [130]:
geo = Nominatim(user_agent = 'Ana')

In [131]:
geo = Nominatim(user_agent = 'Ana')
localizacion = geo.geocode('South Carolina')
localizacion

Location(South Carolina, United States, (33.6874388, -80.4363743, 0.0))

In [132]:
df_geopy = pd.DataFrame(lista_estados_ok, columns = ["state_province"])

In [133]:
def sacar_localizacion(estado):
    
    if estado != "unknown":
        return geo.geocode(estado)[-1]
    
    else:
        return np.nan, np.nan

In [134]:
df_geopy[["latitud", "longitud"]] = df_geopy.apply(lambda df: sacar_localizacion(df["state_province"]), axis= 1, result_type = "expand")

In [135]:
df_geopy.head(2)

,state_province,latitud,longitud
0,unknown,NaN,NaN
1,Pennsylvania,40.969989,-77.727883


In [ ]:
#librería sleep from time

___
- Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

In [136]:
df_geopy.shape

(51, 3)

In [137]:
df_universidades.shape

(2491, 5)

In [138]:
df_universidades = pd.merge(left=df_universidades, right=df_geopy, how='left', on='state_province')

df_universidades.sample(2)

,web_pages,alpha_two_code,state_province,name,country,latitud,longitud
2213,https://www.loc.edu/,US,unknown,LeMoyne-Owen College,United States,NaN,NaN
1545,http://www.witcc.edu,US,unknown,Western Iowa Tech Community College,United States,NaN,NaN


In [139]:
df_universidades.shape

(2491, 7)

___
8. Crea una BBDD en mysql que contenga las siguientes tablas:
- Tabla países: donde encontraremos las siguientes columnas:
    - idestado: primary key, integer, autoincremental
    - nombre_pais: varchar
    - nombre_provincia: varchar
    - latitud: decimal
    - longitud: decimal
- Tabla universidades: donde encontraremos las siguientes columnas:
    - iduniversidades: primary key, integer, autoincremental
    - nombre_universidad: varchar
    - pagina_web: varchar
    - paises_idestado: foreing key

Scrip de creación en la siguiente ruta: `src/script_creacion_bbdd_universidades.sql`, con el siguiente esquema:


![bbdd_universidades_esquema](../images/bbdd_universidades_esquema.png)

___
9. Introduce todo el código que habéis ido creando en funciones, siguiendo la misma lógica que hemos seguido en los pairs:


In [140]:
def llamar_api():
    
    """
    Realiza llamadas a una API para obtener información de universidades en diferentes países.
    
    Returns:
        pd.DataFrame: DataFrame con la información de universidades.
    """
    
    df_universidades = pd.DataFrame() #creamos un dataframe vacío en el que ir sumando cada país
    
    paises = ["United States", "Canada", "Argentina"]
    
    for pais in paises:
        response = requests.get(url= f"http://universities.hipolabs.com/search?country={pais}")
        print(f"Resultado de la conexión de {pais}: {response.status_code} {response.reason}")
        df = pd.json_normalize(response.json())
        df_universidades = pd.concat([df_universidades, df], axis = 0)
    return df_universidades

In [141]:
def limpiar(dataframe= df_universidades):
    
    """
    Limpia y transforma el DataFrame recibido.

    Args:
        dataframe (pd.DataFrame): DataFrame con información de universidades.

    Returns:
        pd.DataFrame: DataFrame limpio y transformado.
    """
    
    dataframe.columns = [column.replace("-", "_") for column in dataframe.columns] # Reemplazamos en los nombres de las columnas "-" por "_"
    dataframe.drop("domains", axis= 1, inplace= True) # Eliminamos la columna "domains" (por ser redundante con "web_page")
    dataframe = dataframe.explode("web_pages") # Creamos una fila por cada "web_page"
    dataframe.drop_duplicates(subset= "name", inplace= True) # Eliminamos duplicados por nombre de universidad
    dataframe["name"] = dataframe["name"].str.replace('"', '') # Reemplazamos las comillas en el nombre de la universidad para que no de error la inserción en SQL
    dataframe["state_province"].fillna("unknown", inplace= True) # Reemplazamos los nulos de "state_province" por "unknown"
    
    # Creamos el diccionario que necesitamos para limpiar los nombres de los estados:
    dict_siglas = {'unknown': 'unknown', 'Ontario': 'Ontario', 'British Columbia': 'British Columbia', 'Pennsylvania': 'Pennsylvania', 'Quebec': 'Quebec', 
                   'Buenos Aires': 'Buenos Aires', 'Ciudad Autónoma de Buenos Aires': 'Buenos Aires', 'Alberta': 'Alberta', 'NY': 'New York', 'New Brunswick': 'New Brunswick', 
                   'Nova Scotia': 'Nova Scotia', 'Manitoba': 'Manitoba', 'California': 'California', 'Córdoba': 'Córdoba', 'New York, NY': 'New York', 'Saskatchewan': 'Saskatchewan', 
                   'Texas': 'Texas', 'Entre Ríos': 'Entre Ríos', 'Santa Fé': 'Santa Fé', 'Mendoza': 'Mendoza', 'North Carolina': 'North Carolina', 'TX': 'Texas', 'Ohio': 'Ohio', 
                   'GA': 'Georgia', 'CA': 'California', 'Santiago Del Estero': 'Santiago Del Estero', 'Salta': 'Salta', 'Washington': 'Washington', 'Misiones': 'Misiones', 
                   'Catamarca': 'Catamarca', 'Yukon': 'Yukon', 'Formosa': 'Formosa', 'Jujuy': 'Jujuy', 'La Rioja': 'La Rioja', 'La Pampa': 'La Pampa', 'San Juan': 'San Juan', 
                   'San Luis': 'San Luis', 'Prince Edward Island': 'Prince Edward Island', 'Maine': 'Maine', 'Newfoundland and Labrador': 'Newfoundland and Labrador', 
                   'New York': 'New York', 'Utah': 'Utah', 'NV': 'Nevada', 'Iowa': 'Iowa', 'VA': 'Virginia', 'Indiana': 'Indiana', 'Colorado': 'Colorado', 'IN': 'Indianapolis', 
                   'Illinois': 'Illinois', 'Michigan': 'Michigan', 'New Hampshire': 'New Hampshire', 'South Carolina': 'South Carolina', 'Virginia': 'Virginia', 'Missouri': 'Missouri', 
                   'ND': 'North Dakota', 'MI': 'Michigan', 'Florida': 'Florida', 'Tucumán': 'Tucumán'}
    
    dataframe["state_province"] = dataframe["state_province"].map(dict_siglas, na_action="ignore") # Hacemos un map que reemplace los valores según el diccionario anterior
    
   
    return dataframe

In [142]:
    # Ahora vamos a crear dos nuevas columnas en el dataframe (latitud y longitud):
    
def añadir_lat_long(dataframe= df_universidades):

    """
    Añade las columnas de latitud y longitud al DataFrame usando geopy.

    Args:
        dataframe (pd.DataFrame, optional): DataFrame con información de universidades. Por defecto es df_universidades.

    Returns:
        pd.DataFrame: DataFrame con las columnas de latitud y longitud añadidas.
    """
    
    lista_estados = dataframe["state_province"].unique()    # Sacamos el listado de estados que tenemos en nuestro dataframe

    df_geopy = pd.DataFrame(lista_estados, columns = ["state_province"]) # Creamos nuevo dataframe con estas provincias
    
    geo = Nominatim(user_agent = 'Ana') # Llamamos a geopy
    
    latitud = [] # Creamos dos listas nuevas, que serán nuestras nuevas columnas del dataframe
    longitud = [] 
    
    for estado in df_geopy["state_province"]: # Iteramos por cada estado de la columna "state_province"
        
        if estado == "unknown": # Si el estado es "unknown", queremos que nos devuelva np.nan
            latitud.append(np.nan)
            longitud.append(np.nan)
        
        else:       # Sino, sacamos la latitud y la longitud gracias a geopy
            lat = geo.geocode(estado)[-1][0]
            long = geo.geocode(estado)[-1][1]
            latitud.append(lat)
            longitud.append(long)

    df_geopy["latitud"] = latitud # Añadimos las listas al dataframe
    df_geopy["longitud"] = longitud
    
    # Por último, añadimos estas columnas al dataframe inicial:
    
    dataframe = pd.merge(left=dataframe, right=df_geopy, how='left', on='state_province')
      
    return dataframe

In [143]:
df_universidades2 = llamar_api()

Resultado de la conexión de United States: 200 OK


Resultado de la conexión de Canada: 200 OK
Resultado de la conexión de Argentina: 200 OK


In [144]:
df_universidades2 = limpiar(df_universidades2)

In [145]:
df_universidades2.sample(2)

,web_pages,alpha_two_code,state_province,name,country
52,http://www.unibo.edu.ar/,AR,unknown,Universidad de Bologna - Representación en Bue...,Argentina
1548,http://www.allencc.edu,US,unknown,Allen County Community College,United States


In [146]:
df_universidades2= añadir_lat_long(df_universidades2)

In [147]:
df_universidades2.sample(2)

,web_pages,alpha_two_code,state_province,name,country,latitud,longitud
2124,http://www.westerntc.edu,US,unknown,Western Technical College,United States,NaN,NaN
1023,http://www.umass.edu/,US,unknown,University of Massachusetts at Amherst,United States,NaN,NaN


___
### BONUS

10. Introduce los datos en la BBDD de SQL:

In [148]:
def crear_bbdd(contraseña, nombre_bbdd):

    mydb = mysql.connector.connect(host="localhost",
                                    user="root",
                                    password=f'{contraseña}',
                                    auth_plugin = 'mysql_native_password') 
    mycursor = mydb.cursor()
    print("Conexión realizada con éxito")

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        
    except:
        print("La BBDD ya existe")


In [149]:
crear_bbdd("AlumnaAdalab", "universidades")

Conexión realizada con éxito


In [150]:
def crear_tablas(contraseña, nombre_bbdd):
    
    """
    Crea una base de datos MySQL si no existe.
    """
    
    mydb = mysql.connector.connect(user='root',
                                    password= f'{contraseña}',
                                    host='127.0.0.1', 
                                    database=f"{nombre_bbdd}",
                                    auth_plugin = 'mysql_native_password')
    mycursor = mydb.cursor()
    
    try:
        mycursor.execute("""CREATE TABLE IF NOT EXISTS `paises` (
                            `idestado` INT NOT NULL AUTO_INCREMENT,
                            `nombre_pais` VARCHAR(45) NULL,
                            `nombre_provincia` VARCHAR(45) NULL,
                            `latitud` DECIMAL(10, 7) NULL,
                            `longitud` DECIMAL(11, 7) NULL,
                            PRIMARY KEY (`idestado`))
                            ENGINE = InnoDB;

                            CREATE TABLE IF NOT EXISTS `universidades` (
                            `iduniversidades` INT NOT NULL AUTO_INCREMENT,
                            `nombre_universidad` VARCHAR(100),
                            `pagina_web` VARCHAR(100) NULL,
                            `paises_idestado` INT NOT NULL,
                            PRIMARY KEY (`iduniversidades`),
                            CONSTRAINT `fk_universidades_paises`
                                FOREIGN KEY (`paises_idestado`)
                                REFERENCES `paises` (`idestado`)
                                ON DELETE NO ACTION
                                ON UPDATE NO ACTION)
                            ENGINE = InnoDB;""")
        
    except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)

In [151]:
crear_tablas("AlumnaAdalab", "universidades")

In [152]:
def insertar_paises(dataframe, contraseña, nombre_bbdd):
    
    """
    Inserta información de países en la base de datos.

    Args:
        dataframe (pd.DataFrame): DataFrame con información de universidades.
        contraseña (str): Contraseña para la base de datos MySQL.
        nombre_bbdd (str): Nombre de la base de datos.

    """
    
    df_paises = dataframe[["state_province", "country", "latitud", "longitud"]].drop_duplicates()
    
    for _, fila in df_paises.iterrows():
        
        mydb = mysql.connector.connect(user='root',
                        password=f"{contraseña}",
                        host='127.0.0.1',
                        database=f"{nombre_bbdd}",
                        auth_plugin = 'mysql_native_password')
        
        mycursor = mydb.cursor()
        
        mycursor.execute("SELECT DISTINCT nombre_provincia FROM paises;")
    
        ciudades = mycursor.fetchall()

        if tuple([fila["state_province"],fila["country"]]) not in ciudades:

            if fila["state_province"] != "unknown":
                
                try: 
                    mycursor.execute(f"""INSERT INTO paises (nombre_pais, nombre_provincia, latitud, longitud)
                                VALUES ('{fila["country"]}', '{fila["state_province"]}', {fila["latitud"]}, {fila["longitud"]})""")
                    mydb.commit() 

                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)
            
            else:
                
                try: 
                    mycursor.execute(f"""INSERT INTO paises (nombre_pais, nombre_provincia, latitud, longitud)
                                VALUES ('{fila["country"]}', '{fila["state_province"]}', NULL, NULL)""")
                    mydb.commit() 

                except mysql.connector.Error as err:
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg) 
 

In [153]:
def insertar_universidades(dataframe, contraseña, nombre_bbdd):
    
    """
    Inserta información de universidades en la base de datos.

    Args:
        dataframe (pd.DataFrame): DataFrame con información de universidades.
        contraseña (str): Contraseña para la base de datos MySQL.
        nombre_bbdd (str): Nombre de la base de datos.

    """

    for _, fila in dataframe.iterrows():
        
        mydb = mysql.connector.connect(user='root', password=f'{contraseña}',
                                host='127.0.0.1', database=f'{nombre_bbdd}',  auth_plugin = 'mysql_native_password')
        mycursor = mydb.cursor()
        
        mycursor.execute("SELECT DISTINCT pagina_web FROM universidades;")
    
        webs = mycursor.fetchall()
        
        lista_webs = [web[0] for web in webs]
        
        if fila["web_pages"] not in lista_webs:
        
            try: 
                mycursor.execute(f"""SELECT idestado
                                FROM paises WHERE nombre_provincia = '{fila["state_province"]}' and nombre_pais = '{fila["country"]}'""")
                idestado = mycursor.fetchall()[0][0]
                
                try: 
                    mycursor.execute(f"""
                            INSERT INTO universidades (nombre_universidad, pagina_web, paises_idestado) 
                            VALUES ("{fila["name"]}", "{fila["web_pages"]}", {idestado});
                            """)
                    mydb.commit() 

                except mysql.connector.Error as err:
                    print('_______')
                    print(f'Error al insertar la universidad {fila["name"]} y la pagina {fila["web_pages"]} perteneciente al estado {fila["state_province"]} en el país {fila["country"]}')
                    print(err)
                    print("Error Code:", err.errno)
                    print("SQLSTATE", err.sqlstate)
                    print("Message", err.msg)
                    
            except: 
                print(f'Sorry, no tenemos el estado {fila["state_province"]} en el país {fila["country"]} en la BBDD y por lo tanto no te podemos dar su id.')

In [154]:
insertar_paises(df_universidades, "AlumnaAdalab", "universidades")

In [155]:
insertar_universidades(df_universidades, "AlumnaAdalab", "universidades")